In [ ]:
# 挂载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

print("云端硬盘已挂载！")
print("查看云端硬盘内容:")
!ls -la "/content/drive/MyDrive/"


In [ ]:
import zipfile
import os

# 1. 查找data.zip文件
print("🔍 寻找data.zip文件...")

possible_paths = [
    "/content/drive/MyDrive/data.zip",
    "/content/drive/MyDrive/我的云端硬盘/data.zip",
    "/content/drive/MyDrive/我的云端硬碟/data.zip",
]

found = False
for path in possible_paths:
    if os.path.exists(path):
        print(f"找到文件: {path}")
        # 复制到Colab工作目录
        !cp "{path}" /content/
        found = True
        break

if not found:
    print("没找到data.zip，请检查云端硬盘路径")
    print("当前云端硬盘目录内容:")
    !ls -la "/content/drive/MyDrive/" | head -20

In [ ]:
# 解压data.zip
if os.path.exists('/content/data.zip'):
    print("正在解压data.zip...")
    with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("解压完成！")

    # 检查目录结构
    print("检查目录结构:")
    !ls -la /content/data/
else:
    print("data.zip不存在，请确认已上传到云端硬盘")

In [ ]:
# 检查数据
import yaml
import os

print("🔍 检查数据完整性...")

# 检查data.yaml
if os.path.exists('/content/data/data.yaml'):
    print("data.yaml存在")
    with open('/content/data/data.yaml', 'r') as f:
        data_cfg = yaml.safe_load(f)
    print(f"类别数 (nc): {data_cfg.get('nc')}")

# 检查图片数量
train_img_count = len(os.listdir('/content/data/images/train'))
val_img_count = len(os.listdir('/content/data/images/val'))
print(f"训练图片: {train_img_count} 张")
print(f"验证图片: {val_img_count} 张")

# 检查标签数量
train_label_count = len(os.listdir('/content/data/labels/train'))
val_label_count = len(os.listdir('/content/data/labels/val'))
print(f"训练标签: {train_label_count} 个")
print(f"验证标签: {val_label_count} 个")

In [ ]:
# 开始训练
from ultralytics import YOLO
import torch

print("开始训练...")
print(f"GPU可用: {torch.cuda.is_available()}")

model = YOLO('yolov8n.pt')  # 使用预训练权重

# 10个epoch
model.train(
    data='/content/data/data.yaml',
    epochs=10,              # 10个epoch
    imgsz=640,              # 图像尺寸
    batch=16,               # batch size
    device=0,               # GPU
    workers=4,              # 数据加载线程
    patience=3,             # 早停
    save=True,
    plots=True,
    verbose=True,
    lr0=0.01,              # 学习率
    cache=False,           # 禁用缓存
)

print("训练完成！")

In [ ]:
# 列出训练结果文件
import os
import glob

# 找到最新的训练目录
train_dirs = sorted(glob.glob('/content/runs/detect/train*'))
if train_dirs:
    latest_train = train_dirs[-1]
    print(f"训练结果目录: {latest_train}")

    # 列出目录内容
    print("\n目录内容:")
    !ls -la "{latest_train}"
else:
    print("没有找到训练结果目录")

In [ ]:
# 显示训练结果图表
from IPython.display import Image, display

if train_dirs:
    # 1. 损失曲线
    loss_plot = f"{latest_train}/results.png"
    if os.path.exists(loss_plot):
        print("损失曲线:")
        display(Image(filename=loss_plot))
    else:
        print("找不到损失曲线图")

    # 2. 验证结果
    val_plot = f"{latest_train}/val_batch0_pred.jpg"
    if os.path.exists(val_plot):
        print("\n验证批次预测示例:")
        display(Image(filename=val_plot))

    # 3. 混淆矩阵
    conf_matrix = f"{latest_train}/confusion_matrix.png"
    if os.path.exists(conf_matrix):
        print("\n混淆矩阵:")
        display(Image(filename=conf_matrix))

    # 4. PR曲线
    pr_curve = f"{latest_train}/PR_curve.png"
    if os.path.exists(pr_curve):
        print("\nPR曲线:")
        display(Image(filename=pr_curve))

In [ ]:
# 训练日志和指标
import pandas as pd
import yaml

if train_dirs:
    # 读取训练参数
    args_file = f"{latest_train}/args.yaml"
    if os.path.exists(args_file):
        with open(args_file, 'r') as f:
            args = yaml.safe_load(f)
        print("训练参数:")
        print(f"- 轮次: {args.get('epochs')}")
        print(f"- 批次大小: {args.get('batch')}")
        print(f"- 图像尺寸: {args.get('imgsz')}")
        print(f"- 学习率: {args.get('lr0')}")

    # 查看结果CSV文件
    results_csv = f"{latest_train}/results.csv"
    if os.path.exists(results_csv):
        print("\n训练指标表格:")
        df = pd.read_csv(results_csv)
        # 显示关键列
        key_columns = ['epoch', 'train/box_loss', 'train/cls_loss', 'val/box_loss', 'val/cls_loss', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
        available_cols = [col for col in key_columns if col in df.columns]
        print(df[available_cols].tail(10))  # 显示最后10个epoch

    # 查看最佳模型信息
    weights_dir = f"{latest_train}/weights"
    if os.path.exists(weights_dir):
        print(f"\n🏆 模型权重文件:")
        !ls -la "{weights_dir}"

        # 检查最佳模型大小
        best_model = f"{weights_dir}/best.pt"
        if os.path.exists(best_model):
            size_mb = os.path.getsize(best_model) / (1024 * 1024)
            print(f"最佳模型大小: {size_mb:.2f} MB")

In [ ]:
from ultralytics import YOLO
import torch

print("优化参数重新训练")

# 使用预训练权重
model = YOLO('yolov8n.pt')

# 优化后的训练参数
model.train(
    data='/content/data/data.yaml',
    epochs=30,              # 增加到30个epoch
    imgsz=640,
    batch=16,
    device=0,
    workers=4,
    patience=10,            # 增加早停耐心
    save=True,
    plots=True,
    verbose=True,
    lr0=0.001,             # 减小学习率
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,       # 增加预热
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,               # 调整损失权重
    cls=0.5,
    dfl=1.5,
    hsv_h=0.015,           # 减少数据增强
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,           # 关闭旋转增强
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,            # 保持mosaic
    mixup=0.0,             # 关闭mixup
    copy_paste=0.0,
    erasing=0.0,
    crop_fraction=1.0,
)

print("重新训练完成！")

In [ ]:
# 强化训练：设置40个epoch和多阶段学习率
from ultralytics import YOLO
import torch

print(" 强化训练：设置40个epoch+多阶段学习率")

# 加载最佳模型
model = YOLO('/content/runs/detect/train6/weights/best.pt')

# 多阶段训练函数
def train_in_stages():
    """分阶段训练，每个阶段使用不同策略"""

    # 阶段1：采用10个epoch稳定训练
    print("阶段1: 稳定训练 (10 epochs)")
    model.train(
        data='/content/data/data.yaml',
        epochs=10,
        imgsz=640,
        batch=16,
        device=0,
        lr0=0.0003,          # 小学习率稳定训练
        mosaic=1.0,
        mixup=0.0,           # 第一阶段不用mixup
        degrees=5.0,
        save=False,          # 暂时不保存
        verbose=True,
        name='stage1',
        exist_ok=True,
    )

    # 阶段2：采用15个epoch数据增强增强
    print("阶段2: 增强数据增强 (15 epochs)")
    model.train(
        resume=True,         # 继续训练
        epochs=25,           # 总共25个epoch（再训练15个）
        lr0=0.0005,          # 稍微提高学习率
        mosaic=1.0,
        mixup=0.2,           # 加入mixup
        degrees=10.0,
        scale=0.5,
        shear=0.1,
        save=True,
        verbose=True,
        name='stage2',
        exist_ok=True,
    )

    # 阶段3：再采用15个epoch精细训练
    print("阶段3: 精细微调 (15 epochs)")
    model.train(
        resume=True,         # 继续训练
        epochs=40,           # 总共40个epoch（再训练15个）
        lr0=0.0001,          # 极小的学习率微调
        mosaic=1.0,
        mixup=0.3,           # 增强mixup
        degrees=15.0,
        scale=0.7,
        shear=0.2,
        perspective=0.0005,
        save=True,
        verbose=True,
        name='stage3',
        exist_ok=True,
        patience=20,         # 更大耐心
    )

# 开始分阶段训练
train_in_stages()

print("✅ 40个epoch强化训练完成！")

In [ ]:
# 强化训练：设置40个epoch
from ultralytics import YOLO
import time

print("分阶段强化训练（共40个epoch）")

# 阶段1：10个epoch
print("="*50)
print("阶段1： 10个epoch")
print("="*50)

model = YOLO('/content/runs/detect/train6/weights/best.pt')  # 从现有最佳模型开始

# 第一阶段训练
model.train(
    data='/content/data/data.yaml',
    epochs=10,              # 训练10个epoch
    imgsz=640,
    batch=16,
    device=0,
    lr0=0.0003,             # 小学习率稳定训练
    mosaic=1.0,
    mixup=0.0,              # 第一阶段不用mixup
    degrees=5.0,
    save=True,
    verbose=True,
    name='stage1_continued',
    exist_ok=True,
    patience=10,
)

print("第一阶段完成！")

# 阶段2：加载第一阶段的最佳模型，继续训练
print("\n" + "="*50)
print("阶段2: 15个epoch")
print("="*50)

# 加载第一阶段训练的最佳模型
model = YOLO('/content/runs/detect/stage1_continued/weights/best.pt')

# 第二阶段训练
model.train(
    data='/content/data/data.yaml',
    epochs=15,              # 再训练15个epoch
    imgsz=640,
    batch=16,
    device=0,
    lr0=0.0005,             # 稍微提高学习率
    mosaic=1.0,
    mixup=0.2,              # 加入mixup
    degrees=10.0,
    scale=0.5,
    shear=0.1,
    save=True,
    verbose=True,
    name='stage2_enhanced',
    exist_ok=True,
    patience=15,
)

print("第二阶段完成！")

# 阶段3：加载第二阶段的最佳模型，
print("\n" + "="*50)
print("阶段3: 15个epoch")
print("="*50)

# 加载第二阶段训练的最佳模型
model = YOLO('/content/runs/detect/stage2_enhanced/weights/best.pt')

# 第三阶段训练
model.train(
    data='/content/data/data.yaml',
    epochs=15,              # 再训练15个epoch
    imgsz=640,
    batch=16,
    device=0,
    lr0=0.0001,             # 极小的学习率微调
    mosaic=1.0,
    mixup=0.3,              # 增强mixup
    degrees=15.0,
    scale=0.7,
    shear=0.2,
    perspective=0.0005,
    save=True,
    verbose=True,
    name='stage3_finetune',
    exist_ok=True,
    patience=20,
)

print("全部分阶段训练完成！共40个epoch")
print("最终模型保存在: /content/runs/detect/stage3_finetune/weights/best.pt")

In [ ]:
# 验证代码
from ultralytics import YOLO
import os
import glob

# 找到最新的训练目录（按修改时间排序）
train_dirs = sorted(glob.glob('/content/runs/detect/*'), key=os.path.getmtime, reverse=True)

if train_dirs:
    latest_train = train_dirs[0]  # 取最新的一个
    best_model_path = f"{latest_train}/weights/best.pt"

    print(f"验证最新模型: {best_model_path}")

    if os.path.exists(best_model_path):
        # 加载模型并验证
        model = YOLO(best_model_path)
        results = model.val(
            data='/content/data/data.yaml',
            imgsz=640,
            batch=8,
            device=0,
            split='val',
            verbose=True
        )

        print(f"\n最终验证结果:")
        print(f"mAP50: {results.box.map50:.4f}")
        print(f"mAP50-95: {results.box.map:.4f}")

        # 与之前对比
        old_map = 0.134
        new_map = results.box.map50
        improvement = ((new_map - old_map) / old_map * 100) if old_map > 0 else 0
        print(f"相比之前(0.134)提升: {improvement:.1f}%")

        # 判断效果
        if new_map > 0.15:
            print("训练有效果，mAP超过0.15")
        elif new_map > 0.134:
            print("略有提升，但效果有限")
        else:
            print("效果没有提升，可能训练出现问题")

    else:
        print("找不到最佳模型文件")
        print(f"请检查路径是否存在: {best_model_path}")

        # 列出目录内容
        print(f"\n目录内容:")
        if os.path.exists(latest_train):
            for item in os.listdir(latest_train):
                item_path = os.path.join(latest_train, item)
                if os.path.isdir(item_path):
                    print(f"  [文件夹] {item}")
                    # 显示子文件夹内容
                    if item == 'weights':
                        for weight_file in os.listdir(item_path):
                            print(f"      {weight_file}")
                else:
                    print(f"  [文件] {item}")
else:
    print("没有找到训练目录")
    print("当前 /content/runs/detect/ 目录内容:")
    !ls -la /content/runs/detect/

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

print("生成测试图片展示...")

# 加载最终模型
model = YOLO('/content/runs/detect/stage3_finetune/weights/best.pt')

# 3张有代表性的验证集图片
test_images = [
    '/content/data/images/val/batch_1_000000.jpg',
    '/content/data/images/val/batch_1_000001.jpg',
    '/content/data/images/val/batch_1_000002.jpg'
]

# 确保图片存在，如果不存在就找其他图片
available_images = []
for img in test_images:
    if os.path.exists(img):
        available_images.append(img)
    else:
        # 找一张存在的图片
        for f in os.listdir('/content/data/images/val')[:5]:
            if f.endswith(('.jpg', '.png', '.jpeg')):
                available_images.append(f'/content/data/images/val/{f}')
                break

# 预测并可视化
for i, img_path in enumerate(available_images[:3]):
    print(f"处理: {os.path.basename(img_path)}")
    results = model.predict(img_path, conf=0.3, save=False)

    # 绘制结果
    annotated = results[0].plot()
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.title(f'测试图片 {i+1}: {os.path.basename(img_path)}', fontsize=12)
    plt.axis('off')

    # 保存图片
    output_path = f'/content/test_result_{i+1}.png'
    cv2.imwrite(output_path, annotated)
    print(f"保存到: {output_path}")
    plt.show()

print("测试图片生成完成！")

In [ ]:
# 生成训练前后对比图表
import matplotlib.pyplot as plt
import numpy as np

print("生成性能对比图...")

# 训练前后的关键指标对比
metrics = ['mAP50', '召回率(R)', '精确率(P)']
before = [0.134, 0.179, 0.318]
after = [0.168, 0.209, 0.329]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, before, width, label='训练前', color='lightcoral', alpha=0.8)
bars2 = ax.bar(x + width/2, after, width, label='训练后', color='lightgreen', alpha=0.8)

# 添加数值标签
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom', fontsize=10)

autolabel(bars1)
autolabel(bars2)

ax.set_xlabel('评估指标', fontsize=12)
ax.set_ylabel('数值', fontsize=12)
ax.set_title('训练前后性能对比 (提升25.4%)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/training_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("对比图已保存: /content/training_comparison.png")

In [ ]:
from google.colab import files
import glob
import zipfile
import os
import shutil

print("打包下载所有结果文件...")

# 创建打包文件夹
output_dir = '/content/final_submission'
os.makedirs(output_dir, exist_ok=True)

# 需要复制的文件
key_files = {
    '模型文件': '/content/runs/detect/stage3_finetune/weights/best.pt',
    '训练参数': '/content/runs/detect/stage3_finetune/args.yaml',
    '训练曲线': '/content/runs/detect/stage3_finetune/results.png',
    '验证结果': '/content/runs/detect/val2/confusion_matrix.png',
    '项目结构': '/content/project_structure.txt',
    '性能对比图': '/content/training_comparison.png',
    '配置数据': '/content/data/data.yaml'
}

copied_items = []  # 记录成功复制的文件

for desc, src_path in key_files.items():
    if os.path.exists(src_path):
        filename = os.path.basename(src_path)
        dest_path = os.path.join(output_dir, filename)
        try:

            shutil.copy2(src_path, dest_path)
            copied_items.append((desc, filename))
            print(f"已添加: {desc} ({filename})")
        except Exception as e:
            print(f"复制失败 [{desc}]: {e}")
    else:
        print(f"文件不存在，已跳过: {desc} ({src_path})")

# 创建README文件
readme_content = """垃圾检测模型 - 最终提交文件
=====================================
... (这里保持你原有的readme_content内容不变) ...
"""
readme_path = os.path.join(output_dir, 'README.txt')
with open(readme_path, 'w') as f:
    f.write(readme_content)
copied_items.append(('说明文档', 'README.txt'))
print(f"已添加: 说明文档 (README.txt)")

# 创建ZIP包
zip_path = '/content/garbage_detection_final.zip'
try:
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for item in os.listdir(output_dir):
            full_path = os.path.join(output_dir, item)
            zipf.write(full_path, arcname=item)
    print(f"\n打包完成！文件大小: {os.path.getsize(zip_path)/1024/1024:.2f} MB")
except Exception as e:
    print(f"\n创建ZIP包失败: {e}")

# 下载ZIP文件
print("\n准备下载最终打包文件...")
try:
    # 显式确认调用的是正确的模块
    from google.colab import files as colab_files
    colab_files.download(zip_path)
    print("下载指令已发送到浏览器。")
except Exception as e:
    print(f"下载失败: {e}")
    print(f"你可以手动从左侧文件栏找到并下载: {zip_path}")

In [25]:
!git clone https://github.com/xingyiliang86-rgb/garbage-detection-yolov8.git
%cd garbage-detection-yolov8

Cloning into 'garbage-detection-yolov8'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 8.30 KiB | 2.07 MiB/s, done.
/content/garbage-detection-yolov8


In [29]:
# 方法1：直接保存当前笔记本（最简单）
from google.colab import drive

# 保存到Google Drive（如果有挂载）
# 或者直接用这个方法强制保存
!pip install -q ipynbname

import ipynbname
nb_path = ipynbname.path()
print(f"当前笔记本路径: {nb_path}")

# 如果没有保存过，手动保存
if str(nb_path) == 'Not running in a notebook':
    print("笔记本未保存，正在保存...")
    # 手动指定一个名字
    notebook_name = "garbage_detection_training.ipynb"
    !jupyter nbconvert --to notebook --output "{notebook_name}" --output-dir "/content" /dev/null 2>/dev/null || true
    !echo '{"cells": [], "metadata": {}, "nbformat": 4, "nbformat_minor": 4}' > "/content/{notebook_name}"
    print(f"已创建: /content/{notebook_name}")
else:
    print(f"笔记本已存在: {nb_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.6 MB/s eta 0:00:00
当前笔记本路径: /fileId=1-NqTck6v-2ePAgSotvq5r6EKzNccCvD5
笔记本已存在: /fileId=1-NqTck6v-2ePAgSotvq5r6EKzNccCvD5


In [ ]:
# 快速保存并推送脚本
import os
import subprocess
import json

print("🔍 快速查找和推送笔记本")
print("=" * 50)

# 1. 先检查所有可能的ipynb文件
print("搜索.ipynb文件...")
!find /content -name "*.ipynb" 2>/dev/null || echo "未找到.ipynb文件"

# 2. 如果没有，创建当前笔记本
current_notebook = "garbage_detection_colab.ipynb"
if not os.path.exists(f'/content/{current_notebook}'):
    print(f"创建笔记本: {current_notebook}")

    # 获取当前笔记本内容
    try:
        # 尝试通过Colab API获取
        from google.colab import _message
        notebook_content = _message.blocking_request('get_notebook', timeout_sec=10)
        with open(f'/content/{current_notebook}', 'w') as f:
            json.dump(notebook_content, f)
    except:
        # 如果失败，创建基本结构
        basic_nb = {
            "cells": [{
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": [
                    "# 垃圾检测训练Colab笔记本\n",
                    "# 作者: xingyiliang86-rgb\n",
                    "# 创建时间: 自动生成\n"
                ]
            }],
            "metadata": {
                "colab": {
                    "name": "垃圾检测训练",
                    "provenance": []
                },
                "kernelspec": {
                    "display_name": "Python 3",
                    "name": "python3"
                }
            },
            "nbformat": 4,
            "nbformat_minor": 4
        }
        with open(f'/content/{current_notebook}', 'w') as f:
            json.dump(basic_nb, f)

    print(f"已创建: /content/{current_notebook}")

# 3. 克隆GitHub仓库
repo_url = "https://github.com/xingyiliang86-rgb/garbage-detection-yolov8.git"
repo_name = "garbage-detection-yolov8"

print(f"克隆仓库: {repo_url}")
!git clone {repo_url} 2>/dev/null || echo "仓库可能已存在"

# 4. 进入仓库并准备
os.chdir(f'/content/{repo_name}')
!git pull origin main

# 5. 确保notebooks目录存在
!mkdir -p notebooks

# 6. 复制所有找到的ipynb文件
import glob
ipynb_files = glob.glob('/content/*.ipynb') + glob.glob('/content/**/*.ipynb', recursive=True)

if ipynb_files:
    print(f"找到 {len(ipynb_files)} 个笔记本文件:")
    for nb in ipynb_files:
        nb_name = os.path.basename(nb)
        !cp "{nb}" "notebooks/{nb_name}"
        print(f"复制: {nb_name}")
else:
    # 使用我们创建的那个
    !cp "/content/{current_notebook}" "notebooks/"
    print(f"使用创建的文件: {current_notebook}")

# 7. 推送
print("推送到GitHub...")
!git config --global user.name "xingyiliang86-rgb"
!git config --global user.email "xingyiliang86@gmail.com"
!git add notebooks/
!git commit -m "Add Colab training notebook(s)"
!git push origin main

print("=" * 50)
print("完成！")
print(f"查看: https://github.com/xingyiliang86-rgb/garbage-detection-yolov8/tree/main/notebooks")